In [1]:
import kfp
from kfp.components import load_component_from_url, create_component_from_func
from kfp.components import InputPath, OutputPath

import sys
sys.path.insert(0, "..")
from constants import NAMESPACE, HOST
from utils.auth import get_session_cookie

In [5]:
from typing import NamedTuple
from kfp.components import InputPath, OutputPath, create_component_from_func

# Ref: https://www.kubeflow.org/docs/components/pipelines/sdk/pipelines-metrics/
def produce_metrics() -> NamedTuple('Outputs', [
  ('mlpipeline_metrics', 'Metrics'),
]):
    import json

    accuracy = 0.8
    metrics = {
        'metrics': [{
            'name': 'accuracy-score', # The name of the metric. Visualized as the column name in the runs table.
            'numberValue':  accuracy, # The value of the metric. Must be a numeric value.
            'format': "PERCENTAGE",   # The optional format of the metric. Supported values are "RAW" (displayed in raw format) and "PERCENTAGE" (displayed in percentage format).
        }]
    }
    return [json.dumps(metrics)]

produce_metrics_op = create_component_from_func(
    produce_metrics,
    base_image='python:3.7',
    packages_to_install=[],
    output_component_file='../components/visualize_metrics/component.yaml',
)

In [3]:
# Define a pipeline and create a task from a component:
def my_pipeline(url):
    produce_metrics_op()

In [4]:
session_cookie = get_session_cookie()
client = kfp.Client(
    host=f"{HOST}/pipeline",
    cookies=f"authservice_session={session_cookie}",
    namespace=NAMESPACE,
)
client.create_run_from_pipeline_func(
    my_pipeline,
    arguments={}
)

RunPipelineResult(run_id=f6a2e21f-c395-4e3a-9629-aa25b61fb659)